# Date pre-processing 데이터 전처리
- 데이터 형식에 대한 처리
    - 공백 문자
        - str.strip() : 양쪽 공백 제거
        - str.lstrip() : 왼쪽 공백 제거
        - str.rstip() : 오른쪽 공백 제거
    - 데이터 타입
    - 불규칙한 대소문자
    - 불규칙한 구분기호
    - 유효하지 않은 문자
    - 불규칙한 날짜 및 시간 표기

In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

## 1. 날짜 형식

### 1.1. 날짜 데이터: str 타입

In [2]:

str_date = ['2022/11/01', '2022.10.31', '2021-10-09']

In [3]:
# Series로 저장
pd.Series(str_date)

0    2022/11/01
1    2022.10.31
2    2021-10-09
dtype: object

<br>

1. str 타입을 datetime 타입으로 변환 후 pandas 저장
- `pd.to_datetime()`

In [4]:
pd.to_datetime(str_date)

DatetimeIndex(['2022-11-01', '2022-10-31', '2021-10-09'], dtype='datetime64[ns]', freq=None)

<br>

2. str 타입으로 저장 후 data type 변경   

- `Series.dtype`: dtype 확인

- `Series.astype()`: dtype 변경

In [5]:
pd.Series(str_date).astype('datetime64')

0   2022-11-01
1   2022-10-31
2   2021-10-09
dtype: datetime64[ns]

<br>

### 1.2. 날짜 데이터: `timestamp` 타입

- `timestamp`: 기준시각(1970.01.01 00:00:00 UTC)로부터 몇 초가 경과했는지 표기 =`UNIX 시간`

In [6]:
stamp_date = [1234000, 1234101, 1234202, 1234300, 123450]

pd.Series(stamp_date)

0    1234000
1    1234101
2    1234202
3    1234300
4     123450
dtype: int64

<br>

- `datetime` 타입으로 변환 후 저장
- `timestampe`의 기본 `unit=ns`(nano seconds) 나노세컨즈: 10억분의 1초
- optional parameter `unit` default: `ns`, days: `D`, seconds :`s`, milli seconds: `ms`, micro seconds: `us`

In [7]:
pd.Series(pd.to_datetime(stamp_date))

0   1970-01-01 00:00:00.001234000
1   1970-01-01 00:00:00.001234101
2   1970-01-01 00:00:00.001234202
3   1970-01-01 00:00:00.001234300
4   1970-01-01 00:00:00.000123450
dtype: datetime64[ns]

<br>

- `unit='s'`: seconds

In [8]:
pd.Series(pd.to_datetime(stamp_date, unit='s'))

0   1970-01-15 06:46:40
1   1970-01-15 06:48:21
2   1970-01-15 06:50:02
3   1970-01-15 06:51:40
4   1970-01-02 10:17:30
dtype: datetime64[ns]

<br>

- `unit='D'`: days 

In [9]:
pd.Series(pd.to_datetime(stamp_date, unit='D'))
# 2038년 문제

OutOfBoundsDatetime: cannot convert input 1234000 with the unit 'D'

<br>

- `unit='ms'`

In [ ]:
pd.Series(pd.to_datetime(stamp_date, unit='ms'))

<br>

- `unit-='us'` : default

In [ ]:
pd.Series(pd.to_datetime(stamp_date, unit='us'))

## 2. label 형식 통일

- data encoding 작업에 포함

<br>

- `map()`

- dict 타입으로 encoding map을 생성해서 적용

- gender: `0`=남자, `1`=여자

In [ ]:
gender_df = pd.DataFrame({'gender': [0, 0, 0, 1, 0, 1]})
gender_map = {0:'M', 1:'F'}

gender_df

<br>

- `gender_df` 변수의 `gender` column값을 `map`함수를 이용해 `0`은 `M`으로, `1`은 `F`로 변환

In [ ]:
gender_df['gender'].map(gender_map)

<br>

- Python 내장함수 `replace()`도 사용 가능

In [ ]:
gender_df['gender'].replace(0, "M").replace(1, "F")

In [ ]:
gender_df['gender'].replace([0, 1], ['M', 'F'])

In [ ]:
gender_df['gender'].replace({0:'M', 1:'F'})

<br>

## 3. 문자 형식(대소문자, 기호 등) 통일

In [ ]:
my_data = {'Name':['Jane', 'Albert', 'John'],
          'Age':[18, 19, 21]}

my_df = pd.DataFrame(my_data)
my_df

<br>

- column name을 소문자로 바꾸는 방법: `str.lower()`

- 빈 list에 소문자로 변경한 column을 모두 적재한 뒤 대입

In [ ]:
my_df1 = my_df.copy()

new_cols = list()

for col in my_df.columns:
    print(col.lower())
    new_cols.append(col.lower())

In [ ]:
new_cols

In [ ]:
my_df1.columns = new_cols

my_df1

<br>

- `df.columns.str.upper()`

In [ ]:
my_df2 = my_df.copy()

my_df2.columns = my_df2.columns.str.upper()
my_df2

<br>

- 내부 item ('Name' column)의 모든 data를 소문자로 통일
- column `age`: 정수 자료 > exclude
- `.apply([함수명])`: 해당 함수의 return value로 column 내부 data를 일괄적으로 교체

In [ ]:
def change_lower(value):
    return value.lower()

In [ ]:
my_df2['NAME'].apply(change_lower)

<br>

- `.apply()`는 `map`으로 대체 가능

In [ ]:
print(my_df2)

my_df2['NAME'].map(change_lower)

<br>

- `AGE` column의 value가 20 이상이면 "성인", 19 이하면 "미성년자" return하는 `is_adult()`함수 정의 후 `.apply()`로 적용

In [ ]:
def is_adult(value) -> str :
    if value >= 20:
        return "성인"
    else:
        return "미성년자"    

In [ ]:
my_df2['AGE'].map(is_adult)

## 4. data value에 대한 처리

- 결측값(NaN)

- 이상치(예측 범위 밖의 value)

- 단순 중복 data

- 동일한 의미, 다른 명칭의 중복 data

- 중복 속성
    
    - 다중공선성: 통계학의 회귀분석에서 독립변수들 간에 강한 상관관계가 나타나는 문제

- 불규칙한 data 수집(differs in step, unit)

<br>

- data accumulation

In [ ]:
this_sample = pd.read_csv('data/csv_exam_nan.csv')

this_sample

### 4.1. 결측치 처리 - 삭제, 선택

1. 결측치`NaN`가 하나라도 있는 레코드 삭제

2. 모든 값이 결측인 레코드 삭제

3. 결측치가 하나라도 있는 데이터만 선택

<br>

1. 결측치가 하나 이상인 레코드(row) 삭제

- `df.dropna(how='any')`

- `df.dropna()` parameter default: `how='any'` > 하나라도 `NaN`이면 row 삭제

- optional parameter: `inplace=True or False` > 원본 data에 반영 여부 결정

In [ ]:
this_sample.dropna()

<br>

2. 모든 값이 `NaN`인 record만 삭제: `how='all'`

In [ ]:
this_sample.dropna(how='all')

<br>

3.결측치가 하나라도 있는 data만 선택: `df[df.isnull().any(axis=1)]` > 조건색인 활용

- `df.isnull()`: `NaN`인 cell은 `True`, 아닌 셀은 `False`로 DataFrame 출력

In [ ]:
this_sample.isnull()

<br>

- `df.any(axis=0)`: 해당 column에 `isnull()`의 결과값이 `True`인 셀이 있는지 체크

- default: `axis=0`: column 단위 기준

- `axis=1`: row 단위 기준

In [ ]:
this_sample.isnull().any()

In [ ]:
this_sample.isnull().any(axis=1)

In [ ]:
# 컬럼별 결측치 개수를 세는 법
this_sample.isnull().sum()

In [ ]:
this_sample[this_sample.isnull().any(axis=1)]

#### 결측치 처리 - 대체값
- 연속형 : 임의값(0 등...), mean, median, 예측값, 도메인지식 활용
- 명목형 : mode(최빈값), 예측값, 도메인지식 활용

In [ ]:
# 연속형- 임의의 값으로 대체
# df.fillna(v)
sample = this_sample

sample.fillna(0)

In [ ]:
sample

In [ ]:
# mean - 1) 전체 데이터의 평균값
# sample : 6 * 3 모든 데이터에 대한 평균
# df.mean() : 컬럼별로 평균값
# NaN를 없는 값으로 가정하고 구한 평균
sample.mean()

In [ ]:
# df.values.mean() : Array타입의 연산으로 NaN값이 하나라도 있다면 최종결과도 NaN
# sample.values.mean() -> 전체 평균을 구할 수 있지만 NaN
# df.fillna(0) 으로 결측치를 보완하고 => mean()평균을 구해야 함
# NaN가 0으로 대체된 평균을 먼저 구한다음, 그 평균값으로 결측치를 메꿔주세요
tot_avg = sample.fillna(0).values.mean()
sample.fillna(tot_avg)

In [ ]:
# mean - 2) 결측치가 존재하는 속성(변수 = 컬럼)의 평균값
# sample : math/en/sc 컬럼에 존재하는 결측치 -> math/en/sc 컬럼의 평균값
# df.mean() : 컬럼별 평균
# 인덱싱으로 과목별 평균 조회
print(sample.mean()[0])
print(sample.mean()[1])
print(sample.mean()[2])

In [ ]:
# 컬럼 지정 후 평균 구하기
sample['math'].mean()

In [ ]:
# fillna를 이용해 수학, 영어, 과학 컬럼에 각각 컬럼의 평균값을 결측치 대신 넣어보세요
sample['math'].fillna(sample['math'].mean())

In [ ]:
sample['english'].fillna(sample['english'].mean())

In [ ]:
sample['science'].fillna(sample.mean()[2])

In [ ]:
sample

In [ ]:
# median - 전체 데이터에 대한 중위값
# tot_avg : 54.5
# median() 을 이용해 몇인지 구해서 1:1로 보내주세요.
# Series나 DataFrame은 벡터함수 연산시 자동으로 NaN를 배제합니다.
# 따라서 전체데이터의 중위값을 구하기 위해 먼저 Series로 바꿔줍니다.
# sample.size 를 .reshape()에 넣어주면 자동으로 1차원 배열로 크기 맞춰서 바꿔줍니다.
tot_med = pd.Series(sample.values.reshape(sample.size)).median()
tot_med

In [ ]:
sample.median()

In [ ]:
sample.fillna(tot_med)

In [ ]:
# mode - 범주형 데이터에서는 최빈값을 사용
# describe()
# value_counts()
# collections 라이브러리의 Counter 모듈
df = pd.DataFrame({'label':['A', 'B', 'B', 'C', 'C', 'C', 'D']})
df

In [ ]:
# descirbe()는 통계분석에 사용합니다.
# count 로우는 총 데이터의 개수
# unique는 데이터의 유형이 몇 개인지
# top은 가장 많이 나온 요소가 무엇인지
# freq는 top 빈도가 몇 번인지
df.describe()

In [ ]:
# Series로 변경하기 위해 label 컬럼을 지정한 후, value_counts()를 이용해
# 각 범주별 개수를 구할 수 있음
df['label'].value_counts()

In [ ]:
from collections import Counter
# 1. 라이브러리를 가져오기
# 2. Counter()를 이용해 Counter 타입의 자료 생성
# 3. counter객체.most_common() => [(value1, count1), (value2, count2) ...]

In [ ]:
colors = ['red', 'blue', 'pink', 'blue', 'blue', 'red']

In [ ]:
counter = Counter(colors)

In [ ]:
counter

In [ ]:
# most_common()
# 갯수순으로 나열해줌
counter.most_common()

In [ ]:
# 데이터프레임에도 Counter를 쓸 수 있습니다.
df = pd.DataFrame({'label': ['A', 'B', 'B', 'C', 'C', 'C', 'D']})
df

In [ ]:
Counter(df['label']).most_common()

### 데이터 단위 통일

#### 표준화(Standardization)
- 평균을 기준으로 얼마나 떨어져 있는지를 파악
- sklearn에서 제공하는 전처리 클래스
    - preprocessing 클래스
        - scaler() : 전체 자료의 분포를 평균 0, 표준편차 1이 되도록 스케일링
        - minmax_scale() : 최대/최소값이 각각 1, 0이 되도록 스케일링
        - StandardScaler() : scaler() 함수와 동일한 동작

- 표준화 : (요소값(하나의 데이터) - 평균) / 표준편차
- 삼성전자 vs 작은회사 주식 시세
- 몸무게 vs 키
    - 표준화 결과 : 몸무게 음수, 키 양수
    - 해석 : 몸무게는 평균 이하, 키는 평균 이상(=>마른몸)

In [ ]:
# 전처리 기능을 제공하는 scikitlearn 라이브러리 및 모듈 가져오기
from sklearn.preprocessing import scale, minmax_scale

In [ ]:
np.arange(9)

In [ ]:
# -3이상, 5이하의 정수를 값으로 가지는 9행 1열 배열 생성
x = (np.arange(9)-3).reshape(-1, 1)
x

In [ ]:
# 데이터프레임 생성
# 3개 컬럼 : x, scale(x), minmax_scale(x)
type(x)

In [ ]:
scale(x)

In [ ]:
# np.hstack: 가로 방향으로 붙여줌
# np.vstack: 세로 방향으로 붙여줌
# 9 * 1 data 3개를 horizontal 방향으로 붙여서 9 * 3으로 만들고
# 붙여 준 column별로 수행 한 연산을 column name으로 할당

df = pd.DataFrame(np.hstack([x, scale(x), minmax_scale(x)]),
                 columns=['x', 'scale(x)', 'minmax_scale(x)'])

df

In [ ]:
df['x'].std()